In [4]:
import datetime
import requests
import json
import re
import time
from urllib.parse import quote_plus

def extract_json_block(html, block_id):
    prefix = re.compile(r"AF_init[dD]ata[cC]all[bB]ack\s*\({[^{}]*key:\s*'" + re.escape(block_id) + ".*?data:")
    suffix = re.compile(r"}\s*\)\s*;")
    try:
        block = prefix.split(html)[1]
        block = suffix.split(block)[0]
    except IndexError:
        print("Could not extract block %s" % block_id)
        return '[]'
    block = block.strip()
    block = re.sub(r"^function\s*\([^)]*\)\s*{", "", block)
    block = re.sub("}$", "", block)
    block = re.sub(r", sideChannel: {$", "", block)
    return block

In [5]:

PLAYSTORE_URL = 'https://play.google.com'

def get_app_details(app_id, country='us', lang='en'):
    url = PLAYSTORE_URL + '/store/apps/details?id='
    url += quote_plus(app_id)
    url += '&hl=' + lang
    url += '&gl=' + country
    try:
        result = requests.get(url).text
    except ConnectionError:
        print ('Could not connect to : {0}'.format(url))

    page_blocks = {
        'version': 'ds:3',
        'pricing': 'ds:4',
        'info': 'ds:6',
        'rating': 'ds:7',
        'similar_apps': 'ds:8',
        'content_rating': 'ds:11',
        'reviews': 'ds:18' #ds:24 on desktop
        }

    try:
        pricing = json.loads(extract_json_block(result,page_blocks['pricing']))
        info = json.loads(extract_json_block(result, page_blocks['info']))
        version = json.loads(extract_json_block(result,page_blocks['version']))
        content_rating = json.loads(extract_json_block(result, page_blocks['content_rating']))
        rating = json.loads(extract_json_block(result,page_blocks['rating']))
        similar_apps = json.loads(extract_json_block(result,page_blocks['similar_apps']))
        reviews_block = json.loads(extract_json_block(result,page_blocks['reviews']))    
    except json.JSONDecodeError:
        print(str(je))

    app = {}
    app['link']=url
    app['id']=app_id
    app['title']=info[0][0][0]
    app['description']=info[0][10][0][1]
    
    app['img_src']=info[0][12][1][3][2]
    app['developer_name']=info[0][12][5][1]
    app['developer_link']=PLAYSTORE_URL + info[0][12][5][5][4][2]
    app['published_timestamp']=info[0][12][8][0]
    app['published']=datetime.datetime.fromtimestamp(int(info[0][12][8][0])).strftime('%c')
    app['num_downloads_approx']=info[0][12][9][1]
    app['num_downloads']=info[0][12][9][2]
    
    if info[0][12][12] :
        app['price_inapp']=info[0][12][12][0] 
    else :
        app['price_inapp']='No inapp purchases'
        
    app['category']=info[0][12][25]
        
    app['content_rating']=content_rating[1][2][9][0]
    app['filesize']=version[0]
    app['android_minimum_os']=version[2]
    
    pricing_block = pricing[0][2][0][0][0][1][0]
    if pricing_block[0] == 0 :
        app['paid_app_price'] = "Free"
    else:
        app['paid_app_price'] = pricing_block[2] +" "+ pricing_block[1]
    
    app['rating'] = rating[0][6][0][1]
    
    rating_block=rating[0][6][1]
    if len(rating_block) > 0:
        app['total_ratings_1_star'] = rating_block[1][1]
        app['total_ratings_2_star'] = rating_block[2][1]
        app['total_ratings_3_star'] = rating_block[3][1]
        app['total_ratings_4_star'] = rating_block[4][1]
        app['total_ratings_5_star'] = rating_block[5][1]

    similar_apps_block=similar_apps[1][1][0][0][0]
    similar_apps=[]
    for similar_no in range(len(similar_apps_block)):
        similar_apps.append(similar_apps_block[similar_no][12][0])
        
    app['similar_apps']=similar_apps
    
    reviews=[]
    for review_no in range(len(reviews_block[0])):
        reviews.append(reviews_block[0][review_no][4])
    
    app["reviews"]=reviews
    
    return app


In [6]:
get_app_details("com.miniclip.eightballpool", country="us", lang="en")




{'link': 'https://play.google.com/store/apps/details?id=com.miniclip.eightballpool&hl=en&gl=us',
 'id': 'com.miniclip.eightballpool',
 'title': '8 Ball Pool',
 'description': 'Play 8 Ball Pool, compete with friends and billiard legends in this multiplayer challenge to become the best in 8 ball pool! Play the hit Miniclip 8 Ball Pool game on your mobile and become a pro player of our league in this 3D free online pool competition.<br><br>COMPETE IN PVP OR IN 8 BALL POOL MULTIPLAYER TOURNAMENTS<br>Refine your 8 Ball Pool match competition skills in the practice arena, take on the world in PvP competitions, or enter a 3D multiplayer match in our free online tournaments to win trophies and exclusive pool cues! Challenge your friends in a PvP match and climb levels in our 8 Ball Pool leaderboard!<br><br>PLAY FOR POOL COINS AND WIN EXCLUSIVE ITEMS<br>Customize your cue and pool table! In every competitive PvP match you play in our league, there’ll be Pool Coins at stake – win the match and t

In [7]:
def get_app_ids_for_developer(
    developer_id,
    num=60,
    country='us',
    lang='en',
    ):
    PLAYSTORE_URL = 'https://play.google.com'
    try:
        developer_id = int(developer_id)
        url = PLAYSTORE_URL + '/store/apps/dev?id='
    except ValueError:
        url = PLAYSTORE_URL + '/store/apps/developer?id='

    url += quote_plus(str(developer_id))

    url += '&hl=' + lang
    url += '&gl=' + country

    try:
        result = requests.get(url).text
        apps_block = extract_json_block(result, 'ds:3')
        apps_block = json.loads(apps_block)
        
        #print(data)
    except (json.JSONDecodeError):
        print('Could not parse Play Store response')
 	
    app_packages=[]
    for app in apps_block[0][1][0][0][0]:
      	app_packages.append(app[12][0])
    
    return app_packages


In [8]:
get_app_ids_for_developer(5933611429942957630)

['com.miniclip.eightballpool',
 'com.miniclip.realsniper',
 'com.miniclip.footballstrike',
 'com.miniclip.basketballstars',
 'com.miniclip.topskater',
 'games.onebutton.golfbattle',
 'com.miniclip.plagueinc',
 'com.miniclip.minifootball',
 'com.miniclip.agar.io',
 'com.hypgames.ultimategolf']